In [1]:
# standart libs
import sys
import random
import copy
from operator import itemgetter
from PIL import Image


# 3rd party libs
import pandas as pd
import numpy as np
import gempy as gp
import matplotlib.pyplot as plt
import scipy.stats as ss
from mpl_toolkits.axes_grid1 import make_axes_locatable
from skimage import measure  # type: ignore



# local
import functions.realization_setup as real_setup
import functions.realization_run as real_run
import functions.post_processing as post_pro
import functions.uq_runs as uq_runs

# executable
print(sys.executable)

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


/home/namur/coding/notebooks/env/bin/python3


In [2]:
# instantiate the geo_model
geo_model = gp.create_model("GeoModel")

# defautl data
geo_model = gp.init_data(
    geo_model,
    extent=[0, 1, 0, 1, 0, 1],
    resolution=[1, 1, 1]
)

# compile theno function
gp.set_interpolation_data(
    geo_model,
    compile_theano=True,
    theano_optimizer='fast_run',
)

Active grids: ['regular']
Setting kriging parameters to their default values.
Compiling theano function...
Level of Optimization:  fast_run
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!
Kriging values: 
                     values
range              1.73205
$C_o$            0.0714286
drift equations        [3]


In [3]:
# meta
geo_model_extent_1 = [0,1000,0,1000,0,1000]
section_1 = {
    'p1': [0, 500],
    'p2': [1000, 500],
    'resolution': [200, 200]
}

# sereis
series_df_1 = pd.DataFrame(columns=['name', 'isfault', 'order_series'])
series_df_1.loc[0] = { 'order_series': 0, 'name': 'Basement_Series', 'isfault': False }
series_df_1.loc[1] = { 'order_series': 1, 'name': 'Strat_Series', 'isfault': False }

# surfaces
surfaces_df_1 = pd.DataFrame(columns=['name', 'serie', 'order_surface'])
surfaces_df_1.loc[0] = { 'name': 'basement', 'serie': 'Basement_Series', 'order_surface': 0 }
surfaces_df_1.loc[2] = { 'name': 'rock1', 'serie': 'Strat_Series', 'order_surface': 1 }
surfaces_df_1.loc[1] = { 'name': 'rock2', 'serie': 'Strat_Series', 'order_surface': 2 }

# geoData
surface_points_input_data_1 = pd.read_csv('./data/model2_surface_points.csv')
orientaions_input_data_1 = pd.read_csv('./data/model2_orientations.csv')

# Format geological_input_data
surface_points_original_df_1 = surface_points_input_data_1[['X', 'Y', 'Z', 'formation']]

# rename colums
surface_points_original_df_1.columns = ['X', 'Y', 'Z', 'surface']

# add distribution type and parameter
surface_points_original_df_1['param1'] = 10

# Orientaions
orientations_original_df_1 = orientaions_input_data_1[['X', 'Y', 'Z', 'dip', 'azimuth', 'polarity', 'formation']]

In [4]:
# %%timeit
# setup model 1
real_setup.setup_realization(
        geo_model=geo_model,
        geo_model_extent=geo_model_extent_1,
        section=section_1,
        series_df=series_df_1,
        surfaces_df=surfaces_df_1,
        surface_points_original_df=surface_points_original_df_1,
        orientations_original_df=orientations_original_df_1
)

Active grids: ['regular']
Active grids: ['regular' 'sections']
HOTFIX in update_series()
HOTFIX in update_surfaces()


In [5]:
if real_run.check_setup_single_realization(geo_model):
    solution = gp.compute_model(model=geo_model, sort_surfaces=False)
Bx = post_pro.compute_boolean_matrix_for_section_surface_top(geo_model)
extent = { 'z_min': geo_model_extent_1[4], 'z_max': geo_model_extent_1[5] }
section_coordinates = post_pro.compute_setction_grid_coordinates(geo_model, extent)
tops_dict = post_pro.get_tops_coordinates(Bx, section_coordinates)

Run realizations setup checks until stable workflow.


In [6]:
mapping_object = real_setup.creat_mapping_object(
    series_df=series_df_1,
    surfaces_df=surfaces_df_1
)

In [ ]:
# Copy geological input data to manipulate per realization.
surface_points_copy = copy.deepcopy(surface_points_original_df_1)
contour_lst = {}

for real_i in range (100): 
    
        # manipulate surface_points_copy in place
        uq_runs.manipulate_surface_points_inplace(
            surface_points_copy=surface_points_copy,
            surface_points_original_df=surface_points_original_df_1)
        
        # correct values outside range
        for XYZ_i in ['X', 'Y', 'Z']:

            if XYZ_i == 'X':
                surface_points_copy['X'][surface_points_copy['X'] < geo_model_extent_1[0]] = geo_model_extent_1[0]
                surface_points_copy['X'][surface_points_copy['X'] > geo_model_extent_1[1]] = geo_model_extent_1[1]

            if XYZ_i == 'Y':
                surface_points_copy['Y'][surface_points_copy['Y'] < geo_model_extent_1[2]] = geo_model_extent_1[2]
                surface_points_copy['Y'][surface_points_copy['Y'] > geo_model_extent_1[3]] = geo_model_extent_1[3]

            if XYZ_i == 'Z':
                surface_points_copy['Z'][surface_points_copy['Z'] < geo_model_extent_1[4]] = geo_model_extent_1[4]
                surface_points_copy['Z'][surface_points_copy['Z'] > geo_model_extent_1[5]] = geo_model_extent_1[5]
        
        # Set manipulated surface points
        geo_model.set_surface_points(surface_points_copy, update_surfaces=False)
        gp.map_series_to_surfaces(
            geo_model=geo_model,
            mapping_object=mapping_object
        )

        # update to interpolator
        geo_model.update_to_interpolator()

        # compute realization
        try:
            gp.compute_model(model=geo_model, sort_surfaces=False)
            print(f'Realization {real_i} computed.')
        except:
            print(f'Error in realization {real_i}.')
            pass
        
        # get contours
        # get start and end of section in grid scalar vals array
        arr_len_0, arr_len_n = geo_model.grid.sections.get_section_args('section')

        # CAUTION: if more section present they have to be indexexed accrodingly;
        # get shape of section  # 1st and only one here as only one section present.
        section_shape = geo_model.grid.sections.resolution[0]
        # extract section scalar values from solutions.sections# [series,serie_pos_0:serie_pos_n]
        section_scalar_field_values = geo_model.solutions.sections[1][:,arr_len_0:arr_len_n]

        # number scalar field blocks
        n_scalar_field_blocks = section_scalar_field_values.shape[0]
        # creat a dictionary to assemble all scalat field boolen matrices shifts
        # extract transition towards current level
        contours = {}
        for block_i in range(n_scalar_field_blocks):

            # number scalar field blocks
            n_scalar_field_blocks = section_scalar_field_values.shape[0]
            # creat a dictionary to assemble all scalat field boolen matrices shifts
            # extract transition towards current level
            contours = {}
            for scalar_field_block_i in range(n_scalar_field_blocks):

                # scalarfield values of scalarfield_block-i
                block = section_scalar_field_values[scalar_field_block_i, :]
                # ??? level
                level = geo_model.solutions.scalar_field_at_surface_points[scalar_field_block_i][np.where(
                    geo_model.solutions.scalar_field_at_surface_points[scalar_field_block_i] != 0)]
                # ??? calulcate scalarfeild levels
                levels = np.insert(level, 0, block.max())
                # extract and reshape scalar field values
                scalar_field = block.reshape(section_shape)
                # loop over levels to extract tops present within current scalar field
                for lvl_i in range(len(levels)):

                    # get top name
                    top_name = geo_model.surfaces.df['surface'][lvl_i]
                    # Find contour
                    contour = measure.find_contours(scalar_field, levels[lvl_i])
                    # add contour to contoures if there are some
                    if len(contour) > 0:

                        contours[top_name] = contour[0]
                        contour_lst[f'real_{real_i}_{top_name}'] = contour[0]

Realization 0 computed.
Realization 1 computed.
Realization 2 computed.
Realization 3 computed.
Realization 4 computed.
Realization 5 computed.
Realization 6 computed.
Realization 7 computed.
Realization 8 computed.
Realization 9 computed.
Realization 10 computed.
Realization 11 computed.
Realization 12 computed.
Realization 13 computed.
Realization 14 computed.
Realization 15 computed.
Realization 16 computed.
Realization 17 computed.
Realization 18 computed.
Realization 19 computed.
Realization 20 computed.
Realization 21 computed.
Realization 22 computed.
Realization 23 computed.
Realization 24 computed.
Realization 25 computed.
Realization 26 computed.
Realization 27 computed.
Realization 28 computed.
Realization 29 computed.
Realization 30 computed.
Realization 31 computed.
Realization 32 computed.
Realization 33 computed.
Realization 34 computed.
Realization 35 computed.
Realization 36 computed.
Realization 37 computed.
Realization 38 computed.
Realization 39 computed.
Realizatio

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 20))
for contour in contour_lst.keys():
    ax.plot(
        contour_lst[contour][:,0],
        contour_lst[contour][:,1]   
    )
plt.show()